In [3]:

# don't use 1.7.2 of scipy !!!


import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import copy
from matplotlib import pyplot as plt
import torch.nn.functional as F
import scipy
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigsh
from torch.autograd import Variable, grad
from numpy.linalg import eig as eig
from torch.distributions.multivariate_normal import MultivariateNormal
from utils import *
from models.fc import *
from models.wide_resnet import wide_resnet_t
from models.wide_resnet_1 import WideResNet
from models.all_cnn import allcnn_t
from models.lenet import lenet
import scipy
from scipy.linalg import eigh_tridiagonal
from functions import *
from scipy.optimize import minimize
from dataset import *
from torch.optim.lr_scheduler import CosineAnnealingLR
from backpack import extend, backpack
from backpack.extensions import (
    KFAC,
    KFLR,
    KFRA,
)


%load_ext autoreload
%autoreload 2
# %aimport functions
# %aimport analyze


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# prepare dataset
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {}

# dataset
num_true = 55000
num_prior = 5000
num_random = 0
num_approx = 10000
num_classes = 2
dataset = "mnist"
num_inplanes = 1 if dataset == "mnist" else 3

# fully connected
num_neurons = 600
num_layers = 2

# wide_resnet
depth = 10
num_channels = 4
widen_factor = 8
drop_out = 0

# all_cnn
c1, c2 = 64, 128



model_name = "lenet"
if model_name == "fc":
    args = (num_classes, num_layers, num_neurons)
if model_name == "wr":
    args = (depth, num_inplanes, num_classes, num_channels, widen_factor, drop_out)
    num_classes = 10
if model_name == "all_cnn":
    args = (num_classes, c1, c2)
    num_classes = 10
if model_name == "lenet":
    args = ()
    num_classes = 10
    
path = create_path(model_name, args, num_true, num_random, dataset)
print(path)
mkdir(path)

mc = get_model_class(model_name)
model = mc(*args).to(device)
criterion = nn.CrossEntropyLoss().to(device)
num_params = sum(param.numel() for param in model.parameters())
print(num_params)

if dataset == "mnist":
    train_set, train_set_prior, train_set_approx, test_set = create_mnist(num_classes, num_true, num_prior, num_approx)

if dataset == "cifar10":
    train_set, train_set_prior, train_set_approx, test_set = create_cifar(num_classes, num_true, num_prior, num_approx)

train_loader = torch.utils.data.DataLoader(train_set,
                                          batch_size=500,
                                          shuffle = True,
                                          **kwargs)

train_loader_approx = torch.utils.data.DataLoader(train_set_approx,
                                          batch_size=len(train_set_approx.data),
                                          shuffle = True,
                                          **kwargs)

train_loader_FIM = torch.utils.data.DataLoader(train_set_approx,
                                          batch_size=1,
                                          shuffle = True,
                                          **kwargs)

train_loader_prior = torch.utils.data.DataLoader(train_set_prior,
                                          batch_size=len(train_set_prior.data),
                                          shuffle = True,
                                          **kwargs)

test_loader = torch.utils.data.DataLoader(test_set,
                                         batch_size=500,
                                         shuffle = True,
                                         **kwargs)


./lenet_55000_0_mnist/
44426


In [9]:
##########################
###### training ##########
##########################
epochs = 20
torch.save(model.state_dict(), path + "model_init.pt")


optimizer = optim.Adam(model.parameters(), lr = 2e-3)
scheduler = CosineAnnealingLR(optimizer,T_max=epochs, eta_min = 5e-4)

tr_err, tr_loss = val(model, device, train_loader, criterion)
val_err, val_loss = val(model, device, test_loader, criterion)
print("initial", 'tr_loss', tr_loss, 'val_loss', val_loss,  'tr_err', tr_err, 'val_err', val_err)


for epoch in range(epochs):

    train(model, device, train_loader, criterion, optimizer, epoch)
    
    tr_err, tr_loss = val(model, device, train_loader, criterion)
    val_err, val_loss = val(model, device, test_loader, criterion)

    # using cosine annealing scheduler
    scheduler.step()

    print(epoch, 'tr_loss', tr_loss, 'val_loss', val_loss,  'tr_err', tr_err, 'val_err', val_err)
    print("lr", optimizer.param_groups[0]['lr'])

torch.save(model.state_dict(), path + "model.pt")

initial tr_loss 2.3096226583827626 val_loss 2.309465253353119 tr_err 0.9239272727272727 val_err 0.9274
0 tr_loss 0.22986032949252563 val_loss 0.21880549862980841 tr_err 0.06652727272727277 val_err 0.06420000000000003
lr 0.0019907662554463535
1 tr_loss 0.12806302579966458 val_loss 0.12461539395153523 tr_err 0.037890909090909086 val_err 0.03510000000000002
lr 0.0019632923872213654
2 tr_loss 0.08269446793604981 val_loss 0.08448399938642978 tr_err 0.024945454545454493 val_err 0.025399999999999978
lr 0.001918254893141276
3 tr_loss 0.06489361607554284 val_loss 0.06965646538883448 tr_err 0.019290909090909136 val_err 0.02059999999999995
lr 0.001856762745781211
4 tr_loss 0.048345967229794375 val_loss 0.060224321112036704 tr_err 0.014618181818181863 val_err 0.018100000000000005
lr 0.001780330085889911
5 tr_loss 0.04417338308624246 val_loss 0.05725962612777948 tr_err 0.013472727272727303 val_err 0.018199999999999994
lr 0.0016908389392193551
6 tr_loss 0.036107481728223235 val_loss 0.05053084362298

In [10]:

##################################
##### decay model ################
##################################

if model_name == "fc" or "lenet":

    epochs = 20

    model_init = mc(*args)
    model_init.load_state_dict(torch.load(path + "model_init.pt", map_location='cpu'))
    model_ana = mc(*args)
    model_ana.load_state_dict(torch.load(path + "model.pt", map_location='cpu'))
    model_init, model_ana = model_init.to(device), model_ana.to(device)

    optimizer = optim.Adam(model_ana.parameters(), lr = 1e-3)
    scheduler = CosineAnnealingLR(optimizer,T_max=epochs, eta_min = 1e-5)

    diff = diff_norm(list(model_ana.parameters()), list(model_init.parameters()))
    print("diff_norm_squared", diff**2)

    tr_err, tr_loss = val(model_ana, device, train_loader, criterion)
    val_err, val_loss = val(model_ana, device, test_loader, criterion)
    print("initial", 'tr_loss', tr_loss, 'val_loss', val_loss,  'tr_err', tr_err, 'val_err', val_err)


    for epoch in range(epochs):

        decay = 1.2*optimizer.param_groups[0]['lr']

        train_decay(model_ana, model_init, decay, device, train_loader, criterion, optimizer, epoch) 
        tr_err_true, tr_loss_true = val(model_ana, device, train_loader, criterion)
        val_err, val_loss = val(model_ana, device, test_loader, criterion)

        # using cosine annealing scheduler
        scheduler.step()

        print(epoch, 'tr_loss', tr_loss, 'val_loss', val_loss,  'tr_err', tr_err, 'val_err', val_err)
        print("lr", optimizer.param_groups[0]['lr'], "decay", decay)

    torch.save(model_ana.state_dict(), path + "model_ana.pt")


diff_norm_squared tensor(435.3404, device='cuda:0', grad_fn=<PowBackward0>)
initial tr_loss 0.006344524988989261 val_loss 0.039174803718924525 tr_err 0.0009636363636363887 val_err 0.012700000000000045
207.2321014404297
0 tr_loss 0.006344524988989261 val_loss 0.04249177714809775 tr_err 0.0009636363636363887 val_err 0.013599999999999945
lr 0.0009939057285945933 decay 0.0012
136.79022216796875
1 tr_loss 0.006344524988989261 val_loss 0.05051651634275913 tr_err 0.0009636363636363887 val_err 0.013900000000000023
lr 0.0009757729755661012 decay 0.0011926868743135119
110.39193725585938
2 tr_loss 0.006344524988989261 val_loss 0.054887103475630285 tr_err 0.0009636363636363887 val_err 0.016700000000000048
lr 0.0009460482294732422 decay 0.0011709275706793213
99.6040267944336
3 tr_loss 0.006344524988989261 val_loss 0.05519464053213596 tr_err 0.0009636363636363887 val_err 0.01629999999999998
lr 0.0009054634122155991 decay 0.0011352578753678906
94.70510864257812
4 tr_loss 0.006344524988989261 val_loss

In [15]:
##########################################
##### recap of training statistics #######
##########################################

model = mc(*args)
model.load_state_dict(torch.load(path + "model.pt", map_location='cpu'))
model_init = mc(*args)
model_init.load_state_dict(torch.load(path + "model_init.pt", map_location='cpu'))
model, model_init = model.to(device), model_init.to(device)

num_params = sum(param.numel() for param in model.parameters())
norm_init = norm_2(model_init)
norm_trained = norm_2(model)
diff = diff_norm(list(model.parameters()), list(model_init.parameters()))

tr_err, tr_loss = val(model, device, train_loader, criterion)
val_err, val_loss = val(model, device, test_loader, criterion)
stat_models = dict({"num_params":num_params, "te_tl_ve_vl":(tr_err, tr_loss, val_err, val_loss), "norm_init, norm_trained": (norm_init.item(), norm_trained.item()), "diff_trained": (diff.item())})

if model_name == "fc" or "lenet":
    model_ana = mc(*args)
    model_ana.load_state_dict(torch.load(path + "model_ana.pt", map_location='cpu'))
    model_ana = model_ana.to(device)
    norm_ana = norm_2(model_ana)
    diff_ana = diff_norm(list(model_ana.parameters()), list(model_init.parameters()))
    tr_err_ana, tr_loss_ana = val(model_ana, device, train_loader, criterion)
    val_err_ana, val_loss_ana = val(model_ana, device, test_loader, criterion)
    stat_models = dict({"num_params":num_params, "te_tl_ve_vl":(tr_err, tr_loss, val_err, val_loss), "tea, tla, vea, vla":(tr_err_ana, tr_loss_ana, val_err_ana, val_loss_ana), "norm_init, norm_trained, norm_ana": (norm_init.item(), norm_trained.item(), norm_ana.item()), "diff_trained, diff_ana": (diff.item(), diff_ana.item())})

torch.save(stat_models, path + "stat_models.pt")
print(stat_models)

{'num_params': 44426, 'te_tl_ve_vl': (0.0009636363636363887, 0.006344524861990728, 0.012700000000000045, 0.03917480600066483), 'tea, tla, vea, vla': (0.007490909090909104, 0.028154844096438453, 0.010600000000000054, 0.033069811388850213), 'norm_init, norm_trained, norm_ana': (8.887020111083984, 23.712791442871094, 14.88017749786377), 'diff_trained, diff_ana': (20.86481285095215, 10.549897193908691)}


In [4]:

#################################################
###### kfac of FIM at init and Hess at end ######
###### and their top eigen values and vecs ######
#################################################

if model_name == "fc":
    kfac_list_init = FIM_kfac(model_init, train_loader_prior, mc=500, device="cpu", mode = "kfra", empirical = True)
    eigspace_list_init, eigval_list_init = eigspace_FIM_kron(kfac_list_init)

    kfac_list_end = FIM_kfac(model, train_loader_approx, mc=500, device="cpu", mode = "kfra", empirical = True)
    eigspace_list_end, eigval_list_end = eigspace_FIM_kron(kfac_list_end)

    torch.save((kfac_list_init, eigspace_list_init, eigval_list_init), path + "kfac_all_init.pt")
    torch.save((kfac_list_end, eigspace_list_end, eigval_list_end), path + "kfac_all_end.pt")


    kfac_val_init, kfac_vec_init = kfac_top_eigvec(kfac_list_init, model, 2000)
    kfac_val_end, kfac_vec_end = kfac_top_eigvec(kfac_list_end, model, 2000)

    torch.save((kfac_val_init, kfac_vec_init), path + "kfac_init.pt")
    torch.save((kfac_val_end, kfac_vec_end), path + "kfac_end.pt")

In [ ]:
##############################################################
###### calculating eigen values of ###########################
######  logit Jacobian #######################################
##############################################################

model = mc(*args)
model.load_state_dict(torch.load(path + "model.pt", map_location='cpu'))
model = model.to(device)


k = 1000
FIM_logit, L_logit, u = logit_jacobian(model, 0, criterion, train_loader_FIM, 'cpu', k)
torch.save((FIM_logit, L_logit, u), path + "FIM_logit_end.pt")
torch.save(L_logit, path + "stat_logit.pt")

L_logit_list = []
for i in range(num_classes):
    FIM_logit, L_logit, u = logit_jacobian(model, i, criterion, train_loader_FIM, 'cpu', k)
    L_logit_list.append(L_logit)

torch.save(L_logit_list, path + "stat_logit_all.pt")

In [5]:
##############################################################
###### calculating eigen values of ###########################
######  true FIM at init, true FIM at the end of training ####
######   empirical FIM end ###################################
##############################################################



model = mc(*args)
model.load_state_dict(torch.load(path + "model.pt", map_location='cpu'))
model_init = mc(*args)
model_init.load_state_dict(torch.load(path + "model_init.pt", map_location='cpu'))
model, model_init= model.to(device), model_init.to(device)

FIM_i, L_i, u_i = FIM_true(model_init, criterion, train_loader_FIM, "cpu", 800)
FIM_tt, L_tt, u_tt = FIM_true(model, criterion, train_loader_FIM, "cpu", 800)
FIM_te, L_te, u_te = FIM2(model, criterion, train_loader_FIM, "cpu", 800)

torch.save((FIM_i, L_i, u_i), path + "FIM_true_init.pt")
torch.save((FIM_tt, L_tt, u_tt), path + "FIM_true_end.pt")
torch.save((FIM_te, L_te, u_te), path + "FIM_em_end.pt")


if model_name == "fc" or "lenet":
    model_ana = mc(*args)
    model_ana.load_state_dict(torch.load(path + "model_ana.pt", map_location='cpu'))
    model_ana = model_ana.to(device)
    FIM_tt_ana, L_tt_ana, u_tt_ana = FIM_true(model_ana, criterion, train_loader_FIM, "cpu", 800)
    FIM_te_ana, L_te_ana, u_te_ana = FIM2(model_ana, criterion, train_loader_FIM, "cpu", 800)
    torch.save((FIM_tt_ana, L_tt_ana, u_tt_ana), path + "FIM_true_end_ana.pt")
    torch.save((FIM_te_ana, L_te_ana, u_te_ana), path + "FIM_em_end_ana.pt")



KeyboardInterrupt: 

In [6]:
L, eig, u = torch.load(path + "FIM_true_init.pt")
print(eig.shape)

torch.Size([800])


In [17]:

###############################################
###### calculating eigen values ###############
###### of hess using scipy.sparse #############
###############################################


k = 500
eigenvalues, eigenvectors = hess_scipy(model, k, train_loader_approx, criterion, device)
torch.save((eigenvalues, eigenvectors), path + "eig_hess_scipy.pt")

if model_name == "fc" or "lenet":
    eigenvalues, eigenvectors = hess_scipy(model_ana, k, train_loader_approx, criterion, device)
    torch.save((eigenvalues, eigenvectors), path + "eig_hess_ana_scipy.pt")

vhp iteration tensor(0.0063, device='cuda:0') tensor(0., device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(0.0044, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(0.3171, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(1.3722, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(1.0068, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(0.6625, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(0.7013, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(0.6781, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(0.4989, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(0.3307, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(0.5343, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(0.2887, device='cuda:0')
vhp iteration tensor(0.0063, device='cuda:0') tensor(0.2682, device=

In [ ]:
#######################################################
###### Overlap of FIM at init and end of training #####
#######################################################


FIM_i, L_i, u_i = torch.load(path + "FIM_true_init.pt")
FIM_tt, L_tt, u_tt = torch.load(path + "FIM_true_end.pt")
FIM_te, L_te, u_te = torch.load(path + "FIM_em_end.pt")
eig_hess, u_hess = torch.load(path + "eig_hess_scipy.pt")

over_i_tt = overlap(u_i, u_tt, 500, "cpu")
over_te_hess = overlap(u_te, u_hess, 500, "cpu")

diff = list_to_vec(diff_list(list(model.parameters()), list(model_init.parameters())))
frac_all = proj(diff, u_i, 1000, "cpu")

stat1 = dict({"FIM_init":L_i, "FIM_tt":L_tt, "FIM_te":L_te, "eig_hess":eig_hess})
stat2 = dict({"over_i_tt":over_i_tt, "over_te_hess":over_te_hess})
stat3 = dict({"frac_all":frac_all})


if model_name == "fc":
    FIM_tt_ana, L_tt_ana, u_tt_ana = torch.load(path + "FIM_true_end_ana.pt")
    eig_hess_ana, u_hess_ana = torch.load(path + "eig_hess_ana_scipy.pt")
    kfac_eig_val_init, kfac_eig_vec_init = torch.load(path + "kfac_init.pt")
    kfac_eig_val_end, kfac_eig_vec_end = torch.load(path + "kfac_end.pt")
    over_i_tt_ana = overlap(u_i, u_tt_ana, 500, "cpu")
    over_kfi_i = overlap(kfac_eig_vec_init, u_i, 500, "cpu")
    over_kfe_hess = overlap(kfac_eig_vec_end, u_hess, 500, "cpu")
    stat1 = dict({"FIM_init":L_i, "FIM_tt":L_tt, "FIM_te":L_te, "FIM_tt_ana":L_tt_ana, "eig_hess":eig_hess, "eig_hess_ana":eig_hess_ana , "kfac_eig_val_init": kfac_eig_val_init, "kfac_eig_val_end": kfac_eig_val_end})
    stat2 = dict({"over_i_tt":over_i_tt,"over_i_tt_ana":over_i_tt_ana, "over_te_hess":over_te_hess, "over_kfi_i":over_kfi_i, "over_kfe_hess":over_kfe_hess})
    diff_ana = list_to_vec(diff_list(list(model_ana.parameters()), list(model_init.parameters())))
    frac_all_ana = proj(diff_ana, u_i, 1000, "cpu")
    stat3 = dict({"frac_all":frac_all, "frac_all_ana":frac_all_ana})

torch.save((stat1, stat2, stat3), path + "stat_overlap.pt")


In [42]:
##############################################
####### analytical methods for posterior #####
####### calculation ##########################
##############################################


if model_name == "fc" or "lenet":

    def PB(lbd_bar,eig_hess, epsilon, norm_diff, b, c, delta):

        # eig_hess, epsilon, norm_diff, b, c, delta = args
        gap = np.sum(eig_hess / lbd_bar)

        KL1 = np.sum(np.log(lbd_bar / epsilon) + epsilon / lbd_bar) - len(lbd_bar)
        KL2 = epsilon*(norm_diff**2)
        KL = (KL1 + KL2) / 2
        sec = np.sqrt((KL + 2*np.log(b*np.log(c*epsilon)) + np.log((num_true*np.pi**2) / (6*delta))) / (2*(num_true-1)))
        PB = gap + sec

        return PB


    def cov(eig_hess, epsilon, norm_diff, b, c, delta):
        norm_diff = norm_diff.cpu().detach().numpy()
        eig_hess = eig_hess.cpu().detach().numpy()
        args = (eig_hess, epsilon, norm_diff, b, c, delta)
        lbd_bar = np.ones(len(eig_hess))
        res = minimize(PB, lbd_bar, args = args, method = 'BFGS', tol=1e-6)

        sol = torch.tensor(res.x)
        sol = sol.float()

        return sol


    k = 300

    if model_name == "fc":
        eig_hess, u = torch.load(path + "eig_hess_ana_scipy.pt")
        model_use = model_ana
    if model_name == "lenet":
        eig_hess, u = torch.load(path + "eig_hess_scipy.pt")
        model_use = model

    eig_hess, u = eig_hess[:k], u[:, :k]
    eig_hess, u = ((1 / np.log(2))*eig_hess).to(device), u.to(device)



    norm_diff = diff_norm(list(model_use.parameters()), list(model_init.parameters()))
    mean = list_to_vec(list(model_use.parameters()))

    print("norm_diff" ,norm_diff)


    n_total = 5
    b = 10
    c = 0.05
    delta = 0.025
    mode = "analytic"


    for j in range(60):
        j += 1
        epsilon = (1/c)*np.exp(j/b)

        if mode == "analytic":
            lbd_bar = 4*(num_true-1)*eig_hess + epsilon

        if mode == "numeric":
            lbd_bar = cov(eig_hess, epsilon, norm_diff, b, c, delta)

        if mode == "isotropic":
            lbd_bar = torch.ones(eig_hess.shape) * epsilon
            
        lbd_bar = lbd_bar.to(device)
        
        KL_1 = (torch.sum(torch.log(lbd_bar/epsilon)) + torch.sum(epsilon/lbd_bar) - len(lbd_bar))
        KL_2 = epsilon*(norm_diff**2)
        KL = (KL_1 + KL_2) / 2
        penalty = (np.log((np.pi)**2*(j**2)*num_true/(delta*6)))
        sec = torch.sqrt((KL + penalty) / (2*(num_true-1)))


        tr_err_true_sum, tr_loss_true_sum, val_err_sum, val_loss_sum = 0, 0, 0, 0

        for k in range(n_total):
            # print(j, k)

            r = torch.randn(num_params).to(device)
            r1 = u.T@r
            r1 = (u * (1/torch.sqrt(lbd_bar))) @ r1
            r2_com = u @ (u.T@r)
            r2 = (r - r2_com) / np.sqrt(epsilon)

            param_tuple = tuple(vec_to_list((r1+r2+mean), model_use))
            for (p1, p2) in zip(model_use.parameters(), param_tuple):
                p1.data = p2.data

            tr_err_true, tr_loss_true = val(model_use, device, train_loader, criterion)
            val_err, val_loss = val(model_use, device, test_loader, criterion)

            tr_err_true_sum += tr_err_true
            tr_loss_true_sum += tr_loss_true
            val_err_sum += val_err
            val_loss_sum += val_loss



        tr_err_true_avg = tr_err_true_sum / n_total
        tr_loss_true_avg = (tr_loss_true_sum / n_total)
        val_err_avg = val_err_sum / n_total
        val_loss_avg = val_loss_sum / n_total

        bd = approximate_BPAC_bound(1-tr_err_true_avg, sec.item())


        print(j)
        print('tr_loss_true_avg', tr_loss_true_avg, 'val_loss_avg', val_loss_avg)
        print('tr_err_true_avg', tr_err_true_avg, 'val_err_avg', val_err_avg)

        print(j, "epsilon", epsilon, "bd", bd, "sec", sec.item(), "KL", KL.item())
        print("KL1, KL2, third", KL_1.item(), KL_2.item(), penalty)


        # if j == 37:
        #     stat = dict({"lbd_bar":lbd_bar, "te_tl_ve_vl":(tr_err_true_avg, tr_loss_true_avg, val_err_avg, val_loss_avg), "epsilon_bd_kl_kl1_kl2":(epsilon, bd, KL.item(), KL_1.item(), KL_2.item())})
        #     torch.save(stat, path + "stat_" + mode + ".pt")
        #     print(stat)


832802
norm_diff tensor(8.0422, device='cuda:1', grad_fn=<SqrtBackward>)
300
37
tr_loss_true_avg 0.25375252024693923 val_loss_avg 0.29349191740155217
tr_err_true_avg 0.08788363636363636 val_err_avg 0.09372
37 epsilon 808.946087201348 bd 0.5461233924913831 sec 0.4878810942173004 KL 26160.27734375
KL1, KL2, third 0.0 52320.5546875 22.323504046087738
{'lbd_bar': tensor([808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461,
        808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461,
        808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461,
        808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461,
        808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461,
        808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461,
        808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461,
        808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461, 808.9461,
        808.94